<a href="https://colab.research.google.com/github/FaridRash/IGNIS/blob/main/Notebooks/SNR_weighted_normalized_simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""https://wp.optics.arizona.edu/alumni/wp-content/uploads/sites/113/2024/06/Jacob_Wilson_Master_s_Report_2024.pdf
maybe when you do the calculation cite that the value is taken from this source"""

'https://wp.optics.arizona.edu/alumni/wp-content/uploads/sites/113/2024/06/Jacob_Wilson_Master_s_Report_2024.pdf\nmaybe when you do the calculation cite that the value is taken from\xa0this\xa0source'

In [2]:


"""For the campi flegrei target we are considering a 200 km^2 square area (approximately 14x14km).
But the specific spots we want to investigate are the fumarole (whose vents can reach 10 metres and 165degree)
Consider those would be much smaller than our gsd, so if you consider the signal from that area it would be a
weighted sum of the signal from the fumarola (~10m, 150-160 °C) and from the sourrunding ground (GSD-10m, Ground Temperature)

But this thing of the weighted sum can be implemented later,
for now let’s see what happens with the whole campi flegrei as target,
then let’s consider the hotspot, then let’s consider the weighted sum (if necessary)"""

'For the campi flegrei target we are considering a 200 km^2 square area (approximately 14x14km).\nBut the specific spots we want to investigate are the fumarole (whose vents can reach 10 metres and 165degree)\nConsider those would be much smaller than our gsd, so if you consider the signal from that area it would be a\nweighted sum of the signal from the fumarola (~10m, 150-160 °C) and from the sourrunding ground (GSD-10m, Ground\xa0Temperature)\n\nBut this thing of the weighted sum can be implemented later,\nfor now let’s see what happens with the whole campi flegrei as target,\nthen let’s consider the hotspot, then let’s consider the weighted sum\xa0(if\xa0necessary)'

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider
import pandas as pd

In [4]:
c1 = 3.7418e8
c2 = 1.4388e4
h = 6.626e-34
c = 3e8

In [5]:
def compute_gsd(H, f, p):
  return (H * p / f) **2

In [6]:
Tf = 433.15

In [7]:
def compute_wighted_snr(Tg,Tf, theta_deg, R, tau_a):
  # The area of the target and background
    Af = 10*10
    Agsd = compute_gsd(R, focal_length, pixel)
    Ag = Agsd - Af

    #Blackbody calculation
    cos_theta = np.cos(np.deg2rad(theta_deg))
    M_lambda_g = c1 / (lambda_range**5 * (np.exp(c2 / (lambda_range * Tg)) - 1))
    M_lambda_f = c1 / (lambda_range**5 * (np.exp(c2 / (lambda_range * Tf)) - 1))

    #Radiance calculation
    E_lambda_g = epsilon * M_lambda_g
    E_lambda_f = epsilon * M_lambda_f

    #Signal calculation
    S_lambda_g = ((D**2 * t * cos_theta * Ag) / (4 * R**2 * h * c)) * (lambda_range * E_lambda_g * tau_a * tau_0 * eta * delta_lambda)
    S_lambda_f = ((D**2 * t * cos_theta * Af) / (4 * R**2 * h * c)) * (lambda_range * E_lambda_f * tau_a * tau_0 * eta * delta_lambda)


    #Sum of S
    S_Ag = np.sum(S_lambda_g)
    S_Af = np.sum(S_lambda_f)

    #S' calculation
    S_prime_g = S_Ag / Ag
    S_prime_f = S_Af / Af
    S_prime = S_prime_f - S_prime_g

    #N' calculation
    N_prime = np.sqrt(S_prime_f + S_prime_g)

    #SNR' calculation
    SNR_prime = S_prime / N_prime

    return SNR_prime
    """(Spectral SNR Contribution per λ)
    we dont need this because we are interested in the total SNR'
    and not in the SNR' contribution per λ


    #S'_lambda calculation
    S_prime_lambda_g = S_lambda_g / Ag
    S_prime_lambda_f = S_lambda_f / Af

    #S'_lambda calculation
    S_prime_lambda = S_prime_lambda_f - S_prime_lambda_g

    #SNR'_lambda
    SNR_prime_lambda = S_prime_lambda / N_prime

    """




In [8]:
"""    plt.figure(figsize=(10, 5))
    plt.plot(lambda_range, SNR_prime_lambda, label='SNR Contribution per λ', color='blue')
    plt.xlabel('Wavelength (μm)')
    plt.ylabel('SNR Contribution')
    plt.title(f'Spectral SNR Contribution\nTg = {Tg}K, Tf = {Tf}K, θ = {theta_deg}°, R = {R/1000:.0f}km, τₐ = {tau_a}, Ag = {Ag:.2f}m, Af = {Af:.0f}m' )
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

    print(f"Total SNR' at Tg = {Tg} K, Tf = {Tf} K, θ = {theta_deg}°, R = {R/1000:.0f} km, τₐ = {tau_a}: {SNR_prime:.2f}")"""

'    plt.figure(figsize=(10, 5))\n    plt.plot(lambda_range, SNR_prime_lambda, label=\'SNR Contribution per λ\', color=\'blue\')\n    plt.xlabel(\'Wavelength (μm)\')\n    plt.ylabel(\'SNR Contribution\')\n    plt.title(f\'Spectral SNR Contribution\nTg = {Tg}K, Tf = {Tf}K, θ = {theta_deg}°, R = {R/1000:.0f}km, τₐ = {tau_a}, Ag = {Ag:.2f}m, Af = {Af:.0f}m\' )\n    plt.grid(True)\n    plt.legend()\n    plt.tight_layout()\n    plt.show()\n\n    print(f"Total SNR\' at Tg = {Tg} K, Tf = {Tf} K, θ = {theta_deg}°, R = {R/1000:.0f} km, τₐ = {tau_a}: {SNR_prime:.2f}")'

In [9]:
epsilon = 0.95
tau_0 = 0.89
eta = 0.7
t = 0.01
focal_length = 72.8 / 1000
pixel = 12e-6
F_number = 1.05
D = (focal_length / F_number)

lambda_range = np.arange(8, 12.1, 0.1)
delta_lambda = 0.1

In [10]:
"""
interact(
    compute_wighted_snr,
    Tg=FloatSlider(value=293.15, min=200, max=500, step=1, description='surrounding ground Temperature (K)'),
    Tf=FloatSlider(value=433.15, min=200, max=500, step=1, description='fumarole Temperature (K)'),
    theta_deg=FloatSlider(value=0, min=0, max=90, step=1, description='Theta (°)'),
    R=FloatSlider(value=460000, min=150000, max=460000, step=1000, description='Distance (m)'),
    tau_a=FloatSlider(value=0.85, min=0.0, max=1.0, step=0.01, description='τₐ (Atmosphere)')
)
"""

"\ninteract(\n    compute_wighted_snr,\n    Tg=FloatSlider(value=293.15, min=200, max=500, step=1, description='surrounding ground Temperature (K)'),\n    Tf=FloatSlider(value=433.15, min=200, max=500, step=1, description='fumarole Temperature (K)'),\n    theta_deg=FloatSlider(value=0, min=0, max=90, step=1, description='Theta (°)'),\n    R=FloatSlider(value=460000, min=150000, max=460000, step=1000, description='Distance (m)'),\n    tau_a=FloatSlider(value=0.85, min=0.0, max=1.0, step=0.01, description='τₐ (Atmosphere)')\n)\n"

In [11]:
Tg_range = np.arange(283.15, 373.15, 1)
R_range = np.arange(460000, 150000, -212.30)
tau_a = np.arange(0.1, 0.5, .01)
theta_range = np.arange(0, 90, 1)
Tg_range, R_range, tau_a, theta_range

(array([283.15, 284.15, 285.15, 286.15, 287.15, 288.15, 289.15, 290.15,
        291.15, 292.15, 293.15, 294.15, 295.15, 296.15, 297.15, 298.15,
        299.15, 300.15, 301.15, 302.15, 303.15, 304.15, 305.15, 306.15,
        307.15, 308.15, 309.15, 310.15, 311.15, 312.15, 313.15, 314.15,
        315.15, 316.15, 317.15, 318.15, 319.15, 320.15, 321.15, 322.15,
        323.15, 324.15, 325.15, 326.15, 327.15, 328.15, 329.15, 330.15,
        331.15, 332.15, 333.15, 334.15, 335.15, 336.15, 337.15, 338.15,
        339.15, 340.15, 341.15, 342.15, 343.15, 344.15, 345.15, 346.15,
        347.15, 348.15, 349.15, 350.15, 351.15, 352.15, 353.15, 354.15,
        355.15, 356.15, 357.15, 358.15, 359.15, 360.15, 361.15, 362.15,
        363.15, 364.15, 365.15, 366.15, 367.15, 368.15, 369.15, 370.15,
        371.15, 372.15]),
 array([460000.        , 459787.7       , 459575.4       , ...,
        150466.60000002, 150254.30000002, 150042.00000002]),
 array([0.1 , 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0

In [12]:
print(len(Tg_range) * len(R_range) * len(tau_a) * len(theta_range))

473364000


In [13]:
df = pd.DataFrame(columns=['Tg', 'Tf', 'theta', 'R', 'tau_a'])

In [ ]:
data = []
for i in Tg_range:
    for j in R_range:
        for k in tau_a:
            for l in theta_range:
                # Compute weighted SNR
                #snr = compute_wighted_snr(i, Tf, l, j, k)

                # Append all features + result to the dataset
                data.append({
                    'Tg': i,
                    'Tf': Tf,
                    'theta': l,
                    'R': j,
                    'tau_a': k
                })
                print(len(data))

df = pd.DataFrame(data)

Streaming output truncated to the last 5000 lines.
51969795
51969796
51969797
51969798
51969799
51969800
51969801
51969802
51969803
51969804
51969805
51969806
51969807
51969808
51969809
51969810
51969811
51969812
51969813
51969814
51969815
51969816
51969817
51969818
51969819
51969820
51969821
51969822
51969823
51969824
51969825
51969826
51969827
51969828
51969829
51969830
51969831
51969832
51969833
51969834
51969835
51969836
51969837
51969838
51969839
51969840
51969841
51969842
51969843
51969844
51969845
51969846
51969847
51969848
51969849
51969850
51969851
51969852
51969853
51969854
51969855
51969856
51969857
51969858
51969859
51969860
51969861
51969862
51969863
51969864
51969865
51969866
51969867
51969868
51969869
51969870
51969871
51969872
51969873
51969874
51969875
51969876
51969877
51969878
51969879
51969880
51969881
51969882
51969883
51969884
51969885
51969886
51969887
51969888
51969889
51969890
51969891
51969892
51969893
51969894
51969895
51969896
51969897
51969898
51969899
5196

In [33]:
import pandas as pd

# Initialize an empty list to collect all rows
data = []

# Loop through all combinations
for i in Tg_range:
    for j in R_range:
        for k in tau_a:
            for l in theta_range:
                # Compute weighted SNR
                snr = compute_wighted_snr(i, Tf, l, j, k)

                # Append all features + result to the dataset
                data.append({
                    'Tg': i,
                    'Tf': Tf,
                    'theta': l,
                    'R': j,
                    'tau_a': k,
                    'weighted_SNR': snr
                })

# Convert list to DataFrame
df = pd.DataFrame(data)

# Preview the DataFrame
print(df.head())

# Save to CSV if needed
df.to_csv("snr_dataset.csv", index=False)


Empty DataFrame
Columns: []
Index: []
